In [3]:
import os.path
import pandapower.networks as pn
import pandapower as pp
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
from numpy import flatnonzero as find, ones, zeros, Inf, pi, exp, conj, r_, arange, array
from pypower_.makeYbus import makeYbus
from pypower_.idx_brch import F_BUS, T_BUS, RATE_A, PF, QF, PT, QT, MU_SF, MU_ST
from pypower_.idx_gen import GEN_BUS, PG, QG
from pypower_.idx_cost import MODEL, PW_LINEAR, NCOST, POLYNOMIAL
from pypower_.totcost import totcost
from pypower_.makeSbus import makeSbus
from pypower_.idx_bus import BUS_TYPE, REF, VM, VA, MU_VMAX, MU_VMIN, LAM_P, LAM_Q
import cvxpy as cp
import numpy as np
from scipy.sparse import csr_matrix as sparse
from pypower_.idx_bus import PD, QD
from pypower_.idx_gen import GEN_BUS, PG, QG, GEN_STATUS

In [2]:
import sys, os
print(sys.path)
print(os.path)
path_current = '/home/huzuntao/PycharmProjects/MPP_Powersystem/'
path_ = os.getcwd()
if path_current not in sys.path:
    sys.path.insert(1, '/home/huzuntao/PycharmProjects/MPP_Powersystem/')
elif path_ not in sys.path:
    sys.path.insert(1, path_)


['/home/huzuntao', '/home/huzuntao/.conda/envs/nvenv/lib/python39.zip', '/home/huzuntao/.conda/envs/nvenv/lib/python3.9', '/home/huzuntao/.conda/envs/nvenv/lib/python3.9/lib-dynload', '', '/home/huzuntao/.conda/envs/nvenv/lib/python3.9/site-packages']
<module 'posixpath' from '/home/huzuntao/.conda/envs/nvenv/lib/python3.9/posixpath.py'>


In [4]:
import pandapower as pp
net = pp.create_empty_network()

# create buses
bus1 = pp.create_bus(net, vn_kv=110.)
bus2 = pp.create_bus(net, vn_kv=110.)
bus3 = pp.create_bus(net, vn_kv=110.)
bus4 = pp.create_bus(net, vn_kv=110.)
# bus5 = pp.create_bus(net, vn_kv=110.)

# create 110 kV lines
pp.create_line(net, bus2, bus3, length_km=70., std_type='149-AL1/24-ST1A 110.0')
pp.create_line(net, bus3, bus4, length_km=50., std_type="149-AL1/24-ST1A 110.0")
pp.create_line(net, bus4, bus2, length_km=40., std_type="149-AL1/24-ST1A 110.0")
pp.create_line(net, bus1, bus2, length_km=70., std_type='149-AL1/24-ST1A 110.0')

# create loads
pp.create_load(net, bus2, p_mw=60., controllable=False)
pp.create_load(net, bus3, p_mw=70., controllable=False)
pp.create_load(net, bus4, p_mw=25., controllable=False)

# create generators
eg = pp.create_ext_grid(net, bus1, min_p_mw=0, max_p_mw=1000, vm_pu=1.05)
g0 = pp.create_gen(net, bus3, p_mw=80, min_p_mw=0, max_p_mw=80, vm_pu=1.00, controllable=True)
g1 = pp.create_gen(net, bus4, p_mw=100, min_p_mw=0, max_p_mw=100, vm_pu=1.00, controllable=True)


costeg = pp.create_poly_cost(net, 0, 'ext_grid', cp1_eur_per_mw=20)
costgen1 = pp.create_poly_cost(net, 0, 'gen', cp1_eur_per_mw=10)
costgen2 = pp.create_poly_cost(net, 1, 'gen', cp1_eur_per_mw=10)

/home/huzuntao/.conda/envs/nvenv/lib/python3.9/site-packages/pandapower/auxiliary.py:268: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for item, dtype in list(dtypes.iteritems()):
/home/huzuntao/.conda/envs/nvenv/lib/python3.9/site-packages/pandapower/auxiliary.py:268: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for item, dtype in list(dtypes.iteritems()):
/home/huzuntao/.conda/envs/nvenv/lib/python3.9/site-packages/pandapower/auxiliary.py:268: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for item, dtype in list(dtypes.iteritems()):
/home/huzuntao/.conda/envs/nvenv/lib/python3.9/site-packages/pandapower/auxiliary.py:268: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for item, dtype in list(dtypes.iteritems()):
/home/huzuntao/.conda/envs/nvenv/lib/python3.9/site-

![xx -w50](grid2GUROBI.png)

In [5]:
net.bus["min_vm_pu"] = 0.98
net.bus["max_vm_pu"] = 1.02
net.line["max_loading_percent"] = 50
# net.sn_mva = 100
# net = pp.networks.case14()
# net.gen['vm_pu'] = 1.0
# net.bus["min_vm_pu"] = 0.95
# net.bus["max_vm_pu"] = 1.05
# net.poly_cost[:,3] = 0
# net.poly_cost['cp2_eur_per_mw2'] = 0
net.poly_cost

,element,et,cp0_eur,cp1_eur_per_mw,cp2_eur_per_mw2,cq0_eur,cq1_eur_per_mvar,cq2_eur_per_mvar2
0,0,ext_grid,0.0,20.0,0.0,0.0,0.0,0.0
1,0,gen,0.0,10.0,0.0,0.0,0.0,0.0
2,1,gen,0.0,10.0,0.0,0.0,0.0,0.0


In [7]:
om,ppopt,_ = pp.runopp(net, delta=1e-16)   # solve the OPF problem

In [8]:
# net = pp.networks.case57()
print(net.res_gen)    # output the results of two generators
# net.gen

        p_mw     q_mvar  va_degree     vm_pu
0  79.999577  -2.278173   3.774021  1.019995
1  77.020890 -17.572200   4.755391  1.020000


In [14]:
print(net.res_ext_grid)
net.ext_grid['vm_pu']

       p_mw     q_mvar
0  0.000007  16.144855


0    1.05
Name: vm_pu, dtype: float64

In [15]:
# the total cost of generation
net.res_cost

1570.2048160492504

In [16]:
# the key variable, which represents the information of the OPF problem
om.get_ppc()

{'baseMVA': 1,
 'version': 2,
 'bus': array([[ 0.00000000e+00,  3.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00,  1.05000000e+00,  0.00000000e+00,
          1.10000000e+02,  1.00000000e+00,  1.05000000e+00,
          1.05000000e+00,  1.08190002e+01,  2.18039840e-21,
          0.00000000e+00,  1.65497357e+02],
        [ 1.00000000e+00,  1.00000000e+00,  6.00000000e+01,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00,  1.01080187e+00,  1.05599499e+00,
          1.10000000e+02,  1.00000000e+00,  1.02000000e+00,
          9.80000000e-01,  1.06350842e+01,  7.04471675e-03,
          0.00000000e+00,  0.00000000e+00],
        [ 2.00000000e+00,  2.00000000e+00,  7.00000000e+01,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00,  1.01999500e+00,  3.77402105e+00,
          1.10000000e+02,  1.00000000e+00,  1.02000000e+00,
          9.80000000e-01,  1.016375

In [17]:
## unpack data
ppc = om.get_ppc()
baseMVA, bus, gen, branch, gencost = \
    ppc["baseMVA"], ppc["bus"], ppc["gen"], ppc["branch"], ppc["gencost"]
vv, _, _, _ = om.get_idx()

## problem dimensions
nb = bus.shape[0]          ## number of buses
nl = branch.shape[0]       ## number of branches
ny = om.getN('var', 'y')   ## number of piece-wise linear costs, 'y' is a part of p-w linear costs

## bounds on optimization vars
x0, xmin, xmax = om.getv()

In [18]:
## build admittance matrices
Ybus, Yf, Yt = makeYbus(baseMVA, bus, branch)

/home/ubuntu-h/PycharmProjects/scientificProject/pypower_/makeYbus.py:49: ComplexWarning: Casting complex values to real discards the imaginary part
  tap[i] = branch[i, TAP]  ## assign non-zero tap ratios
/home/ubuntu-h/anaconda3/envs/scientificProject/lib/python3.10/site-packages/scipy/sparse/_coo.py:160: ComplexWarning: Casting complex values to real discards the imaginary part
  self.col = np.array(col, copy=copy, dtype=idx_dtype)


In [57]:
## Set the lower and upper bound for all variables
ll, uu = xmin.copy(), xmax.copy()
# print(f'll : {ll}; uu : {uu}')
ll[xmin == -Inf] = -1e10   ## replace Inf with numerical proxies
uu[xmax ==  Inf] =  1e10
Va_refs = bus[bus[:, BUS_TYPE]  == REF, VA] * (pi / 180)
# print(f"Va_refs: {Va_refs}")
ll[vv["i1"]["Va"]:vv["iN"]["Va"]] = -np.ones_like(bus[:, VA]) * (pi / 2) # Va lower bound 赋值
uu[vv["i1"]["Va"]:vv["iN"]["Va"]] = np.ones_like(bus[:, VA]) * (pi / 2) # Va upper bound 赋值
## deal with the Va_refs
ll[vv["i1"]["Va"]:vv["iN"]["Va"]][bus[:, BUS_TYPE]  == REF] = Va_refs  # Va_refs lower bound 赋值
uu[vv["i1"]["Va"]:vv["iN"]["Va"]][bus[:, BUS_TYPE]  == REF] = Va_refs # Va_refs upper bound 赋值
## deal with reactive power, whose ll and uu is 1e9
ll[ll<-1e4] = -100
uu[uu>1e4] = 100
v_max = uu[vv["i1"]["Vm"]:vv["iN"]["Vm"]][-1]
print(f'll"{ll};\n uu:{uu}')

ll"[ 0.00000000e+00 -1.57079633e+00 -1.57079633e+00 -1.57079633e+00
  1.05000000e+00  9.80000000e-01  9.80000000e-01  9.80000000e-01
 -1.00000000e-16 -1.00000000e-16 -1.00000000e-16 -1.00000000e+02
 -1.00000000e+02 -1.00000000e+02];
 uu:[   0.            1.57079633    1.57079633    1.57079633    1.05
    1.02          1.02          1.02       1000.           80.
  100.          100.          100.          100.        ]


In [20]:
x0[vv["i1"]["Vm"]:vv["iN"]["Vm"]]

array([1.05, 1.  , 1.  , 1.  ])

In [21]:
uu[vv["i1"]["Vm"]:vv["iN"]["Vm"]]

array([1.05, 1.02, 1.02, 1.02])

In [22]:
bus[:, VM] # 赋值

array([1.05      , 1.01080187, 1.019995  , 1.01999964])

In [23]:
x0[vv["i1"]["Vm"]:vv["iN"]["Vm"]] = bus[:, VM] # 赋值

In [24]:
Va_refs = bus[bus[:, BUS_TYPE]  == REF, VA] * (pi / 180)
x0[vv["i1"]["Va"]:vv["iN"]["Va"]] = bus[:, VA] * (pi / 180) # 赋值

In [25]:
## 赋值
x0[vv["i1"]["Pg"]:vv["iN"]["Pg"]] = gen[:,PG]
x0[vv["i1"]["Qg"]:vv["iN"]["Qg"]] = gen[:,QG]


### 不能处理piecewise linear 的cost, 因为没有引入那个函数，（大约在这个位置）


### 处理cost fun

In [26]:
## grab Pg & Qg
Pg = x0[vv["i1"]["Pg"]:vv["iN"]["Pg"]]  ## active generation in p.u.
Qg = x0[vv["i1"]["Qg"]:vv["iN"]["Qg"]]  ## reactive generation in p.u.

##----- evaluate objective function -----
## polynomial cost of P and Q
# use totcost only on polynomial cost in the minimization problem formulation, pwl cost is the sum of the y variables.
ipol = find(gencost[:, MODEL] == POLYNOMIAL)   ## poly MW and MVAr costs
xx = r_[ Pg, Qg ] * baseMVA
if len(ipol)>0:
    f = sum( totcost(gencost[ipol, :], xx[ipol]) )  ## cost of poly P or Q
else:
    f = 0

In [27]:
Pg

array([7.37457879e-06, 7.99995768e+01, 7.70208900e+01])

In [28]:
xx[ipol]

array([7.37457879e-06, 7.99995768e+01, 7.70208900e+01])

In [29]:
First_Or_Con = 4
print(f"{gencost[ipol, First_Or_Con], gencost[:, First_Or_Con]}")

(array([20., 10., 10.]), array([20., 10., 10.]))


In [30]:
gencost

array([[ 2.,  0.,  0.,  2., 20.,  0.],
       [ 2.,  0.,  0.,  2., 10.,  0.],
       [ 2.,  0.,  0.,  2., 10.,  0.]])

### 处理约束部分

In [31]:
## grab Pg & Qg
Pg = x0[vv["i1"]["Pg"]:vv["iN"]["Pg"]]  ## active generation in p.u.
Qg = x0[vv["i1"]["Qg"]:vv["iN"]["Qg"]]  ## reactive generation in p.u.

In [32]:
## put Pg & Qg back in gen
gen[:, PG] = Pg * baseMVA  ## active generation in MW
gen[:, QG] = Qg * baseMVA  ## reactive generation in MVAr

In [33]:

on = find(gen[:, GEN_STATUS] > 0)      ## which generators are on?
gbus = gen[on, GEN_BUS]                   ## what buses are they at?

## form net complex bus power injection vector
nb = bus.shape[0]
ngon = on.shape[0]
## connection matrix, element i, j is 1 if gen on(j) at bus i is ON
Cg = sparse((ones(ngon), (gbus, range(ngon))), (nb, ngon))

## power injected by gens plus power injected by loads converted to p.u.
Sbus = ( Cg * (gen[on, PG] + 1j * gen[on, QG]) - (bus[:, PD] + 1j * bus[:, QD]) ) / baseMVA
# Sbus = makeSbus(baseMVA, bus, gen) ## net injected power in p.u.

In [34]:
Sbus

array([ 7.37457879e-06+16.14485516j, -6.00000000e+01 +0.j        ,
        9.99957682e+00 -2.27817257j,  5.20208900e+01-17.57220036j])

In [35]:
## reconstruct V
Va = x0[vv["i1"]["Va"]:vv["iN"]["Va"]]
Vm = x0[vv["i1"]["Vm"]:vv["iN"]["Vm"]]
V = Vm * exp(1j * Va)
V

array([1.05      +0.j        , 1.01063019+0.01862862j,
       1.01778305+0.06713757j, 1.0164885 +0.08455998j])

In [36]:
Va

array([0.        , 0.01843059, 0.06586909, 0.08299722])

In [37]:
net.res_bus

,vm_pu,va_degree,p_mw,q_mvar,lam_p,lam_q
0,1.050000,0.000000,-0.000007,-16.144855,10.819000,2.180398e-21
1,1.010802,1.055995,60.000000,0.000000,10.635084,7.044717e-03
2,1.019995,3.774021,-9.999577,2.278173,10.163754,-2.693141e-22
3,1.020000,4.755391,-52.020890,17.572200,10.000002,-2.414266e-21


In [38]:
## evaluate power flow equations
mis = V * conj(Ybus * V) - Sbus

In [39]:
ppc['branch']

array([[ 1.00000000e+00+0.j,  2.00000000e+00+0.j,  1.12231405e-03+0.j,
         2.37190083e-03+0.j,  2.32831286e+00+0.j,  4.47735134e+01+0.j,
         2.50000000e+02+0.j,  2.50000000e+02+0.j,  1.00000000e+00+0.j,
         0.00000000e+00+0.j,  1.00000000e+00+0.j, -3.60000000e+02+0.j,
         3.60000000e+02+0.j, -1.81675549e+01+0.j,  3.97820737e+00+0.j,
         1.85594449e+01+0.j, -5.55060515e+00+0.j,  0.00000000e+00+0.j,
         0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
         0.00000000e+00+0.j,  0.00000000e+00+0.j],
       [ 2.00000000e+00+0.j,  3.00000000e+00+0.j,  8.01652893e-04+0.j,
         1.69421488e-03+0.j,  1.66308061e+00+0.j,  4.47735134e+01+0.j,
         2.50000000e+02+0.j,  2.50000000e+02+0.j,  1.00000000e+00+0.j,
         0.00000000e+00+0.j,  1.00000000e+00+0.j, -3.60000000e+02+0.j,
         3.60000000e+02+0.j, -8.55986787e+00+0.j,  3.27243410e+00+0.j,
         8.62951677e+00+0.j, -4.85549795e+00+0.j,  0.00000000e+00+0.j,
         0.00000000e+00+0.

In [40]:
branch[:, RATE_A]

array([44.77351338+0.j, 44.77351338+0.j, 44.77351338+0.j, 44.77351338+0.j])

In [41]:
 ## find branches with flow limits
il = find((branch[:, RATE_A] != 0) & (branch[:, RATE_A] < 1e10))
nl2 = len(il)           ## number of constrained lines

In [42]:
il

array([0, 1, 2, 3])

In [43]:
## 不等式约束： 线电压的功率
ppopt = {}
il = find(branch[:, RATE_A] < 1e10)
nl2 = len(il)           ## number of constrained lines
if any(net.line['max_i_ka']) > 0:
    ppopt['OPF_FLOW_LIM'] = 2

if nl2 > 0:
    flow_max = (branch[il, RATE_A] / baseMVA)**2
    flow_max[flow_max == 0] = 1e5
    print(f"flow_max: {flow_max}")
    if ppopt['OPF_FLOW_LIM'] == 2:      ## current magnitude limit, |I|
        If = Yf * V
        It = Yt * V
        h = r_[ If * conj(If) - flow_max,     ## branch I limits (from bus)
                It * conj(It) - flow_max ].real    ## branch I limits (to bus)
    else:
        ## compute branch power flows
        ## complex power injected at "from" bus (p.u.)
        Sf = V[ branch[il, F_BUS].astype(int) ] * conj(Yf * V)
        ## complex power injected at "to" bus (p.u.)
        St = V[ branch[il, T_BUS].astype(int) ] * conj(Yt * V)
        if ppopt['OPF_FLOW_LIM'] == 1:   ## active power limit, P (Pan Wei)
            h = r_[ Sf.real**2 - flow_max,   ## branch P limits (from bus)
                    St.real**2 - flow_max ]  ## branch P limits (to bus)
        else:                ## apparent power limit, |S|
            h = r_[ Sf * conj(Sf) - flow_max, ## branch S limits (from bus)
                    St * conj(St) - flow_max ].real  ## branch S limits (to bus)
else:
    h = zeros((0,1))

flow_max: [2004.6675+0.j 2004.6675+0.j 2004.6675+0.j 2004.6675+0.j]


In [44]:
## first, the equality constraints (power flow)
# g = r_[ mis.real,            ## active power mismatch for all buses
#         mis.imag ]           ## reactive power mismatch for all buses

## then, the inequality constraints (branch flow limits)
h

array([-1666.13439383, -1923.94759733,   -39.53172446, -1768.24450864,
       -1643.97363903, -1910.43019769,   -75.05058409, -1688.71823828])

In [45]:
If

array([-1.78978229e+01 -4.26626785j, -8.16269557e+00 -3.75370533j,
        4.13605714e+01+15.95114122j,  7.06228866e-06-15.37605253j])

In [46]:
flow_max

array([2004.6675+0.j, 2004.6675+0.j, 2004.6675+0.j, 2004.6675+0.j])

In [47]:
net.res_line

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,-18.167555,3.978207,18.559445,-5.550605,0.391890,-1.572398,0.096571,0.099682,0.099682,1.010802,1.055995,1.019995,3.774021,21.208903
1,-8.559868,3.272434,8.629517,-4.855498,0.069649,-1.583064,0.047156,0.050952,0.050952,1.019995,3.774021,1.020000,4.755391,10.840773
2,43.391373,-12.716703,-42.141643,13.986096,1.249730,1.269394,0.232671,0.230559,0.232671,1.020000,4.755391,1.010802,1.055995,49.504549
3,0.000007,16.144855,0.309198,-17.964304,0.309206,-1.819449,0.080703,0.093294,0.093294,1.050000,0.000000,1.010802,1.055995,19.849862


In [48]:
llv = ll[vv["i1"]["Vm"]:vv["iN"]["Vm"]]**2
uuv = uu[vv["i1"]["Vm"]:vv["iN"]["Vm"]]**2
uuv

array([1.1025, 1.0404, 1.0404, 1.0404])

In [49]:
print(Ybus)

  (0, 0)	(162.99668368338897-343.3133709444148j)
  (1, 0)	(-162.99668368338897+344.47752737211067j)
  (0, 1)	(-162.99668368338897+344.47752737211067j)
  (3, 1)	(-285.24419644593075+602.8356729011938j)
  (1, 1)	(611.2375638127087-1288.7971825456257j)
  (2, 1)	(-162.99668368338897+344.47752737211067j)
  (2, 2)	(391.19204084013353-824.7503689598727j)
  (1, 2)	(-162.99668368338897+344.47752737211067j)
  (3, 2)	(-228.19535715674456+482.268538320955j)
  (3, 3)	(513.4395536026753-1083.607438672254j)
  (2, 3)	(-228.19535715674456+482.268538320955j)
  (1, 3)	(-285.24419644593075+602.8356729011938j)


In [50]:
Pg_lower, Pg_upper = ll[vv["i1"]["Pg"]:vv["iN"]["Pg"]], uu[vv["i1"]["Pg"]:vv["iN"]["Pg"]]
Pg_upper

array([1000.,   80.,  100.])

In [51]:
Pd = bus[:, PD]
Pd

array([ 0., 60., 70., 25.])

In [52]:
baseMVA

1

In [75]:
# Parameters
# obtain the complex load
Pd = bus[:, PD]
Qd = bus[:, QD]
slack_v = net.ext_grid['vm_pu']

lb_update = -1e4

# obtain the range of voltage; the range is from the setting
V_range = v_max - 1
V_ll,V_uu = 1.0-V_range, 1.0+V_range

# obtain the power range
Pg_lower, Pg_upper = ll[vv["i1"]["Pg"]:vv["iN"]["Pg"]], uu[vv["i1"]["Pg"]:vv["iN"]["Pg"]]*baseMVA
Qg_lower, Qg_upper = ll[vv["i1"]["Qg"]:vv["iN"]["Qg"]], uu[vv["i1"]["Qg"]:vv["iN"]["Qg"]]*baseMVA

# the dimension of the OPF
nb = bus.shape[0]          ## number of buses
nl = branch.shape[0]       ## number of branches

ineq_idx = 0

# create the model; set the model type: NonConvex.
model = gp.Model("PowerGeneration");model.setParam('NonConvex', 2)

# creat variables for the model; the numbers in the following codes should be replaced by parameters
V_re = model.addMVar(nb, lb=lb_update, vtype=GRB.CONTINUOUS, name='V real')
V_im = model.addMVar(nb, lb=lb_update, vtype=GRB.CONTINUOUS, name='V imag')
Pg = model.addMVar(ngon, lb=Pg_lower, ub=Pg_upper, vtype=GRB.CONTINUOUS, name='Pg')
Qg = model.addMVar(ngon, lb=Qg_lower, ub=Qg_upper, vtype=GRB.CONTINUOUS, name='Qg')
temp1 = model.addMVar(nb, lb=lb_update, vtype=GRB.CONTINUOUS, name='Temp 1')
temp2 = model.addMVar(nb, lb=lb_update, vtype=GRB.CONTINUOUS, name='Temp 2')
Pg_net = model.addMVar(nb, lb=lb_update, vtype=GRB.CONTINUOUS, name='Pg net')  # the number of buses, too
Qg_net = model.addMVar(nb, lb=lb_update, vtype=GRB.CONTINUOUS, name='Qg net')
Line_temp_1 = model.addMVar(nl, lb=lb_update, vtype=GRB.CONTINUOUS, name='Line temp 1')    # the shape should be line number
Line_temp_2 = model.addMVar(nl, lb=lb_update, vtype=GRB.CONTINUOUS, name='Line temp 2')    # the shape should be line number
Line_temp_3 = model.addMVar(nl, lb=lb_update, vtype=GRB.CONTINUOUS, name='Line temp 3')    # the shape should be line number
Line_temp_4 = model.addMVar(nl, lb=lb_update, vtype=GRB.CONTINUOUS, name='Line temp 4')    # the shape should be line number

# the prameters of branches
G = np.real(Ybus)
B = np.imag(Ybus)
# the parameters for currents in branches
G_yf, B_yf = np.real(Yf), np.imag(Yf)
G_yt, B_yt = np.real(Yt), np.imag(Yt)

# Constraints; insert the formulas
Temp_c1 = model.addConstr(G@V_re - B@V_im == temp1, 'Temp Constr 1')
Temp_c2 = model.addConstr(G @ V_im + B @ V_re == temp2, 'Temp Constr 2')
Pg_net_c = model.addConstr(Cg @ Pg - Pd == Pg_net, 'Pg net Constr')
Qg_net_c = model.addConstr(Cg @ Qg - Qd == Qg_net, 'Qg net Constr')
# print(f'Pd:{Pd}\n Qd:{Qd}')
CVL_CPL = model.addConstrs((V_re[i] * temp1[i] *baseMVA+ V_im[i] * temp2[i]*baseMVA == Pg_net[i]  for i in range(nb)), 'Cvl P')  # break into different parts
CVL_CPL_Q = model.addConstrs((V_im[i] * temp1[i] *baseMVA - V_re[i] * temp2[i]*baseMVA == Qg_net[i] for i in range(nb)), 'Cvl Q')

# the constraints of voltages on all buses; set the constraint for slack bus, then give the upper and lower constraints for other buses
V_slack = model.addConstr(V_re[0]==slack_v, 'V slack real')
V_slack_a = model.addConstr(V_im[0]==0, 'V slack imag')
# the upper&lower constraints for buses except for the slack bus
V_upper = model.addConstrs((V_re[i]**2 + V_im[i]**2 <= V_uu**2 for i in range(1,nb-ineq_idx)), 'V upper bound')  # 对 range 起始值的处理，是处理树状分层搜索的突破口；有一"减"，对应的有一"加"。
# if ineq_idx != 0:
#     V_upper = model.addConstrs((V_re[i]**2 + V_im[i]**2 == V_uu**2 for i in range(ineq_idx,nb)), 'V upper bound')
#
V_lower = model.addConstrs((V_re[i]**2 + V_im[i]**2 >= V_ll**2 for i in range(1,nb)), 'V lower bound')

# Yf * W * Yf; the number is equal to that of the lines/branches
Line_temp_c_1 = model.addConstr(G_yf@V_re - B_yf@V_im == Line_temp_1, 'Branch Con 1')
Line_temp_c_2 = model.addConstr(G_yf @ V_im + B_yf @ V_re == Line_temp_2, 'Branch Con 2')
Line_temp_c_3 = model.addConstr(G_yt@V_re - B_yt@V_im == Line_temp_3, 'Branch Con 3')
Line_temp_c_4 = model.addConstr(G_yt @ V_im + B_yt @ V_re == Line_temp_4, 'Branch Con 4')
# 下式左侧是否应该乘上baseMVA
Line_c_1 = model.addConstrs((Line_temp_1[i]**2 + Line_temp_2[i]**2 <= np.real(flow_max[i]) for i in range(nl)), 'Branch Con 1') # p.u.
Line_c_2 = model.addConstrs((Line_temp_3[i]**2 + Line_temp_4[i]**2 <= np.real(flow_max[i]) for i in range(nl)), 'Branch Con 2') # p.u.

# set the objective of the model
print('ipol:', ipol)
# The "gencost" changes with different cases. Keep in Mind!
# model.setObjective(gencost[ipol, First_Or_Con]@Pg**2+gencost[ipol, First_Or_Con+1]@Pg, GRB.MINIMIZE)
model.setObjective(gencost[ipol, First_Or_Con]@Pg, GRB.MINIMIZE)

# calculate the optimization
# model.optimize()
model.X[:14]

Pg_star, Qg_star = Pg.X, Qg.X
optimized_obj = gencost[ipol, First_Or_Con]@Pg_star


Set parameter NonConvex to value 2
ipol: [0 1 2]
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Core(TM) i5-10600KF CPU @ 4.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 34 rows, 46 columns and 152 nonzeros
Model fingerprint: 0x47ad00e0
Model has 22 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+01, 2e+01]
  Bounds range     [1e-16, 1e+04]
  RHS range        [1e+00, 7e+01]
  QRHS range       [1e+00, 2e+03]
Presolve removed 4 rows and 4 columns

Continuous model is non-convex -- solving as a MIP

Presolve removed 12 rows and 14 columns
Presolve time: 0.00s
Presolved: 85 rows, 50 columns, 292 nonzeros
Presolved model has 11 quadratic constraint(s)
Presolved model has 18 bilinear constraint(s)
Variable types: 50 continuous, 0 integer (0 binary)

In [83]:
len(model.getVars())
len(model.getConstrs())

34

In [55]:
print(Pg)

[7.37457879e-06 7.99995768e+01 7.70208900e+01]


In [56]:
net.res_bus

,vm_pu,va_degree,p_mw,q_mvar,lam_p,lam_q
0,1.050000,0.000000,-0.000007,-16.144855,10.819000,2.180398e-21
1,1.010802,1.055995,60.000000,0.000000,10.635084,7.044717e-03
2,1.019995,3.774021,-9.999577,2.278173,10.163754,-2.693141e-22
3,1.020000,4.755391,-52.020890,17.572200,10.000002,-2.414266e-21


In [85]:
m = gp.Model("model")
# x = m.addMVar(shape=(10), name="x")
y = m.addMVar(shape=(5),lb=np.array([-1,-2,-3,-4,-5]), name="y")

A = np.ones(5)

m.addConstrs((y[i] >=np.array([-1,-2,-3,-4,-5])[i] for i in range(5)), name="const")
m.setObjective(A@Y_.real)
m.optimize()

TypeError: unsupported operand type(s) for *: 'MLinExpr' and 'complex'

In [59]:
m.X

NameError: name 'm' is not defined

In [296]:
for v in m.getVars():
    print(v.varName, v.x)

y[0] -1.0
y[1] -2.0
y[2] -3.0
y[3] -4.0
y[4] -5.0


In [297]:
n = 3
p = 3
np.random.seed(1)
C = np.random.randn(n, n)
A = []
b = []
for i in range(p):
    A.append(np.random.randn(n, n))
    b.append(np.random.randn())

print(f'A ;{A};\n b : {b}')

A ;[array([[-0.24937038,  1.46210794, -2.06014071],
       [-0.3224172 , -0.38405435,  1.13376944],
       [-1.09989127, -0.17242821, -0.87785842]]), array([[ 0.58281521, -1.10061918,  1.14472371],
       [ 0.90159072,  0.50249434,  0.90085595],
       [-0.68372786, -0.12289023, -0.93576943]]), array([[ 0.53035547, -0.69166075, -0.39675353],
       [-0.6871727 , -0.84520564, -0.67124613],
       [-0.0126646 , -1.11731035,  0.2344157 ]])];
 b : [0.04221374671559283, -0.2678880796260159, 1.6598021771098705]


In [298]:
# Define and solve the CVXPY problem.
# Create a symmetric matrix variable
X = cp.Variable((n,n), symmetric=True)
# The operator >> denotes matrix inequality.
constraints = [X >> 0]
constraints += [
    cp.trace(A[i] @ X) == b[i] for i in range(p)
]
prob = cp.Problem(cp.Minimize(cp.trace(C @ X)),
                  constraints)
prob.solve()
# Print result.
print("The optimal value is", prob.value)
print("A solution X is")
print(X.value)

The optimal value is 2.654347058555727
A solution X is
[[ 1.60805504 -0.59770125 -0.69575821]
 [-0.59770125  0.22228555  0.24689067]
 [-0.69575821  0.24689067  1.39679134]]
